In [7]:
import openai
import numpy as np
import pandas as pd
from config import openai_settings, paths

openai.api_key = openai_settings.OPENAI_API_KEY


class RagChatGPT:
    def __init__(self):
        self.prompt = open(
            paths.PROMPT_DIR.joinpath("chatgpt.txt"), encoding="utf-8"
        ).read()
        self.embedidngs = np.load(paths.DATA_DIR.joinpath("embeddings.npy"))
        self.db = self.get_docs()
        
    def get_docs(self):
        db = pd.read_csv(paths.DATA_DIR.joinpath("example.csv"))
        return db['차량 비상시 응급조치'].tolist()

    def query_embedding(self, user_message: str) -> np.ndarray:
        response = openai.Embedding.create(
            model="text-embedding-ada-002",
            input=[user_message],
        )
        query_embedding = np.array(response["data"][0]["embedding"])
        return query_embedding

    def sementic_search(self, user_message: str) -> str:
        query_embedding = self.query_embedding(user_message)
        dot_product = np.dot(self.embedidngs, query_embedding)
        norm_a = np.linalg.norm(query_embedding)
        norm_b = np.linalg.norm(self.embedidngs, axis=1)
        cos_sim = dot_product / (norm_a * norm_b)
        max_index = np.argmax(cos_sim)
        return self.db[max_index]
        

    def reply(self, user_message: str) -> str:
        instruction = self.prompt + "\n\n" + self.sementic_search(user_message)
        print(instruction)
        messages = [{"role": "system", "content": instruction}]

        messages.append({"role": "user", "content": f"{user_message}"})

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
        answer = completion.choices[0].message["content"].strip()
        # answer = answer.replace("\n", " ")
        return answer

In [8]:
chat = RagChatGPT()

In [11]:
chat.reply("엔진과열이 발생했어요")

너는 차량 메뉴얼에 대한 사항을 알려주는 안내원이야.
안내 내용은 차량 비상시 응급조치 중에서 엔진 과열에 대한 내용이야
너는 반드시 안내사항에 있는 내용을 기반으로 답변해야해.

안내사항

차량 비상시 응급조치에 대한 안내사항입니다.
엔진 과열에 대한 내용입니다.

엔진 과열은 기온이 높을 때만 발생하는 것이아니고, 다음과 같은 경우에도 발생할 수 있습니다.
• 냉각수가 부족할 때
• 매우 추울 때
• 냉각수 중에 부동액이 들어있지 않을 때
• 엔진 내부가 얼어 냉각수가 순환하지 못할때 냉각수 중 부동액의 비율이 너무 높거나 낮으면 냉각 성능이 떨어져 엔진이 과열될 수 있습니다.
냉각수 수온계가 장시간 적색 눈금을 가리키면 엔진이 과열되어 엔진 출력이 떨어지고 노킹(Knocking) 현상이 발생합니다.

엔진 과열이 발생하면 다음과 같이 조치하십시오.
1. 비상 경고등을 켜고 도로 가장자리에 안전하게 정차한 후 'P'(주차)로 변속하고 파킹 브레이크를 거십시오.
2. 에어컨이 켜져 있는 경우, 에어컨을 끄십시오.
3. 냉각수나 뜨거운 증기가 냉각수 보조 탱크에서 흘러나오지 않는지 확인하십시오.
4. 냉각수나 뜨거운 증기가 흘러나오지 않으면, 엔진에 시동이 걸린 상태에서 후드를 열고 엔진 내부에 통풍이 잘되도록 하여 엔진을 식히십시오.
• 냉각수나 뜨거운 증기가 흘러나오면, 엔진을 정지시키고 당사 직영 하이테크센터나 블루핸즈에 연락하십시오.
• 엔진에 시동이 걸릴 때 수온계가 떨어지지 않으면 엔진을 정지시키고 엔진이 냉각될때까지 충분히 기다리십시오.
5. 엔진이 충분히 냉각되면 엔진 냉각수의 양 및 누수 여부를 점검하고, 라디에이터 호스 연결 부위, 히터 호스 연결 부위, 워터 펌프등에 누수가 없으면 냉각수를 보충하십시오.
• 냉각수가 누수되었거나, 냉각팬이 회전하지 않거나, 워터 펌프 구동 벨트가 손상되는 등의 이상이 발견되면 운전을 중단하고 당사 직영 하이테크센터나 블루핸즈에서 점검을 받으십시오.
• 냉각수는 부동액과 물을 적정 비율로 혼합하여 사용하십시오

"엔진과열이 발생했을 때는 다음과 같이 조치해야 합니다.\n\n1. 비상 경고등을 켜고 도로 가장자리에 안전하게 정차한 후 'P'(주차)로 변속하고 파킹 브레이크를 걸어주세요.\n2. 에어컨이 켜져 있는 경우, 에어컨을 끄세요.\n3. 냉각수나 뜨거운 증기가 냉각수 보조 탱크에서 흘러나오지 않는지 확인하세요.\n4. 냉각수나 뜨거운 증기가 흘러나오지 않으면, 엔진에 시동이 걸린 상태에서 후드를 열고 엔진 내부에 통풍이 잘 되도록 하여 엔진을 식혀주세요.\n• 냉각수나 뜨거운 증기가 흘러나오면, 엔진을 정지시키고 당사 직영 하이텍센터나 블루핸즈에 연락하세요.\n• 엔진에 시동이 걸릴 때 수온계가 떨어지지 않으면 엔진을 정지시키고 엔진이 냉각될 때까지 충분히 기다리세요.\n5. 엔진이 충분히 냉각되면 엔진 냉각수의 양 및 누수 여부를 점검하고, 라디에이터 호스 연결 부위, 히터 호스 연결 부위, 워터 펌프 등에 누수가 없으면 냉각수를 보충하세요.\n• 냉각수가 누수되었거나, 냉각팬이 회전하지 않거나, 워터 펌프 구동 벨트가 손상되는 등의 이상이 발견되면 운전을 중단하고 당사 직영 하이텍센터나 블루핸즈에서 점검을 받으세요.\n• 냉각수는 부동액과 물을 적정 비율로 혼합하여 사용하세요. 부동액의 비율이 너무 높거나 낮으면 냉각 성능이 떨어져 엔진이 과열될 수 있습니다.\n\n[주의]\n• 냉각수 부족으로 엔진이 과열되었을 때 급하게 차가운 냉각수를 넣으면 엔진에 균열이 생길 수 있으므로 천천히 조금씩 보충하세요.\n• 오버히트가 발생하면 차를 안전하게 정차한 후에 냉각팬이 멈출 때까지 엔진 시동을 끄지 마세요. 수온이 급상승하여 엔진이 고착될 수 있습니다.\n• 냉각팬이 회전하지 않으면 시동을 끄세요."